In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import cv2
import numpy as np
import pandas as pd
import argparse
import webcolors
import colorsys
import matplotlib.pyplot as plt

import skfuzzy as fuzz
from skfuzzy import control as ctrl
import colorsys

In [14]:
index=["color","color_name","hex","R","G","B"]
df_colors = pd.read_csv('colors_prev.csv', names=index, header=None)

In [15]:
colors = {
    "Black": [(0, 0, 0)],
    "White": [(255, 255, 255)],
    "Red": [(255, 0, 0),(139,0,0)],
    "Dark Red":[(139,0,0)],
    "Orange":[(255,69,0),(255,140,0)],
    "Green": [(0, 255, 0),(85,107,47),(128,128,0),(34,139,34),(173,255,47),(124,252,0),(210,214,126)],
    "Blue": [(0, 0, 255),(93, 138, 168),(0,0,128),(65,105,225),(55,67,101)],
    "Light Blue": [(173,216,230),(0,255,255),(178,200,217),(186,192,203),(97,198,224)],
    "Beige":[(240,230,140),(238,232,170),(253,245,230),(230,204,176),(219.55331186, 216.11409801, 207.9812984),(221,192,176),(188,143,119)],
    "Purple":[(128,0,128),(138,43,226),(76,71,134)],
    "Gray":[(128,128,128),(192,192,192),(169,169,169)],
    "Yellow": [(255, 255, 0),(255,215,0),(236,188,43)],
    "Pink": [(255, 0, 255),(220,20,60),(240,128,128),(205,92,92),(226,215,213),(243,219,219),(209,67,121),(247,217,203)],
    "Brown":[(102,51,0),(103,82,76)]}

In [16]:
n=df_colors.shape[0]
ignore_us = ['maya blue']
for i in range(n):
    for key in colors.keys():
        curr_color = str(key).lower()
        if curr_color in df_colors.color_name[i].lower():
            if curr_color in ignore_us:
                continue
            rgb = (df_colors.R[i],df_colors.G[i],df_colors.B[i])
            colors[key].append(rgb)

In [17]:
def my_color_name(rgb,colors):
    min_distance = float("inf")
    closest_color = None
    for color, value in colors.items():
        for val in value:
            distance = sum([(i - j) ** 2 for i, j in zip(rgb, val)])
            if distance < min_distance:
                min_distance = distance
                closest_color = color
    return closest_color

In [18]:
def convert_rgb_to_hsv(red,green,blue):
    #rgb normal: range (0-255, 0-255, 0.255)
    #get rgb percentage: range (0-1, 0-1, 0-1 )
    red_percentage= float(red) / float(255)
    green_percentage= green/ float(255)
    blue_percentage=blue / float(255)
    
    #get hsv percentage: range (0-1, 0-1, 0-1)
    color_hsv_percentage=colorsys.rgb_to_hsv(red_percentage, green_percentage, blue_percentage) 
    
    #get normal hsv: range (0-360, 0-255, 0-255)
    color_h=round(360*color_hsv_percentage[0])
    color_s=round(255*color_hsv_percentage[1])
    color_v=round(255*color_hsv_percentage[2])
    color_hsv=(color_h, color_s, color_h)
    return color_hsv

In [19]:
def get_hsv_value(R,G,B):
    return colorsys.rgb_to_hsv(R,G,B)

In [20]:
df_colors['hsv'] = df_colors.apply(lambda row : convert_rgb_to_hsv(row['R'],
                     row['G'], row['B']), axis = 1) 

# Fuzzy logic models

In [21]:
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import colorsys

In [22]:
"""
Antecedents HSV
HUE: color represented by number from 0(red) - 360(violet)
{ WARM, COOL }

SATURATION: color saturation represented by number from 0(faded/gray color) - 100(full color)
{ GRAY, VERY_FADED, FADED, SATURATED, VERY_SATURATED }

VALUE: brightness represented by number 0(dark) - 100(light)
{ BLACK, VERY_DARK, DARK, BRIGHT, VERY_BRIGHT }
"""

hue_range = np.arange(0, 361, 1)
hue_fuzzy = ['WARM', 'COOL', 'WARM_']
hue = ctrl.Antecedent(hue_range, 'hue')
hue['WARM'] = fuzz.gaussmf(hue.universe, 0, 60)
hue['COOL'] = fuzz.gaussmf(hue.universe, 180, 60)
hue['WARM_'] = fuzz.gaussmf(hue.universe, 360, 60)


sat = ctrl.Antecedent(np.arange(0, 101, 1), 'saturation')
sat_fuzzy = ['GRAY', 'VERY_FADED', 'FADED', 'SATURATED', 'VERY_SATURATED']
sat['GRAY'] = fuzz.gaussmf(sat.universe, 0, 10)
sat['VERY_FADED'] = fuzz.gaussmf(sat.universe, 25, 10)
sat['FADED'] = fuzz.gaussmf(sat.universe, 50, 10)
sat['SATURATED'] = fuzz.gaussmf(sat.universe, 75, 10)
sat['VERY_SATURATED'] = fuzz.gaussmf(sat.universe, 100, 10)


val = ctrl.Antecedent(np.arange(0, 101, 1), 'value')
val_fuzzy = ['BLACK', 'VERY_DARK', 'DARK', 'BRIGHT', 'VERY_BRIGHT']
val['BLACK'] = fuzz.gaussmf(val.universe, 0, 10)
val['VERY_DARK'] = fuzz.gaussmf(val.universe, 25, 10)
val['DARK'] = fuzz.gaussmf(val.universe, 50, 10)
val['BRIGHT'] = fuzz.gaussmf(val.universe, 75, 10)
val['VERY_BRIGHT'] = fuzz.gaussmf(val.universe, 100, 10)

In [23]:
"""
Consequents
TONE: mix of Saturation and Value that indicate if color is neutral or dark/bright
{ NEUTRAL, DARK, BRIGHT }
"""
tone_range = np.arange(0, 12, 1)
tone_fuzzy = ['NEUTRAL', 'DARK', 'BRIGHT']
tone = ctrl.Consequent(tone_range, 'tone')

tone['NEUTRAL'] = fuzz.trapmf(tone.universe, [0, 0, 1, 2])
tone['DARK'] = fuzz.gbellmf(tone.universe, 2, 1, 3)
tone['BRIGHT'] = fuzz.gbellmf(tone.universe, 4, 1, 9.5)
#tone['NEUTRAL_'] = fuzz.trimf(tone.universe, [10, 11, 11])

In [24]:
"""
Fuzzy rules
for tones
"""
rule1 = ctrl.Rule(val['BLACK'] | sat['GRAY'] | sat['VERY_FADED'], tone['NEUTRAL'], 'Dark colors without color (low brightness/dark) considered neutral')
rule2 = ctrl.Rule(val['VERY_DARK'] & sat['SATURATED'], tone['NEUTRAL'], 'Very dark colors with high saturation')
rule3 = ctrl.Rule(val['DARK'] & sat['FADED'], tone['DARK'], 'Dark color with normal saturation')
rule4 = ctrl.Rule(val['DARK'] & sat['VERY_SATURATED'], tone['BRIGHT'], 'Dark color with high saturation')
rule5 = ctrl.Rule(val['BRIGHT'] & sat['SATURATED'], tone['BRIGHT'], 'Bright color with high saturation')
rule6 = ctrl.Rule(val['VERY_BRIGHT'] & sat['FADED'], tone['BRIGHT'], 'Very bright color with some saturation')
rule7 = ctrl.Rule(val['VERY_BRIGHT'] & sat['VERY_SATURATED'], tone['BRIGHT'], 'Very bright color with high saturation')
rule8 = ctrl.Rule(val['VERY_DARK'] & sat['FADED'], tone['NEUTRAL'], 'Very dark color with faded saturation')

In [25]:
"""
Control system
for tones
"""
tone_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8])


In [26]:
"""
Simulation
for tones
"""
tone_sim = ctrl.ControlSystemSimulation(tone_ctrl)

In [27]:
"""
GetMembership
Returns String representing the Fuzzy value given a variable's range, model, and crisp value
"""
def GetMembership(fuzzy_values, var_range, var_model, crisp_value):
    max_membership = 0
    membership_name = fuzzy_values[0]
    for i in range(len(fuzzy_values)):
        temp_memb = fuzz.interp_membership(var_range, var_model[fuzzy_values[i]].mf, crisp_value)
        if temp_memb > max_membership:
            max_membership = temp_memb
            membership_name = fuzzy_values[i]
    return membership_name


## API

In [28]:
"""
Given Saturation and Value, returns a String indicating if the combination 
of both values results in a 'NEUTRAL', 'DARK', or 'BRIGHT' tone.
INPUT:
values - tuple(sat, val)
    + sat - value from 0-100
    + val - value from 0-100
verbose - prints both crisp value and fuzzy value
"""
def GetTone(values, verbose=False):
    tone_sim = ctrl.ControlSystemSimulation(tone_ctrl)
    tone_sim.input['saturation'] = values[0]
    tone_sim.input['value'] = values[1]
    tone_sim.compute()
    tone_output = tone_sim.output['tone']
    tone_membership = GetMembership(tone_fuzzy, tone_range, tone, tone_output)
    if verbose:
        print("TONE CRISP VALUE: ", tone_output)
        print("TONE FUZZY VALUE: ", tone_membership)
    return tone_membership

In [29]:
"""
Given Hue, returns a String indicating if the color belongs
to 'WARM' or 'COOL' colors.
INPUT:
hue - value from 0-360
verbose - prints both crisp value and fuzzy value
"""
def GetColorTemp(hue_val, verbose=False):
    temp_membership = GetMembership(hue_fuzzy, hue_range, hue, hue_val)
    if verbose:
        print("TEMP. CRISP VALUE: ", hue_val)
        print("TEMP. FUZZY VALUE: ", temp_membership)
    return temp_membership

In [30]:
"""
Given Hue, Saturation, and Value, returns a String describing
the specified color. The output is composed of both the tone of
the color, and the temperature of the color.
INPUT:
hsv - tuple(hue, sat, val)
    + hue - value from 0-360
    + sat - value from 0-100
    + val - value from 0-100
OUTPUT: (TONE, TEMP) ex. (DARK, WARM)
"""
def GetColorDesc(hsv):
    tone = GetTone((hsv[1], hsv[2]))
    temp = GetColorTemp(hsv[0])
    if temp == "WARM_": temp = "WARM"
    return (tone, temp)

In [31]:
df_colors['tone'] = df_colors['color_name']
df_colors['color'] = df_colors.apply(lambda row : my_color_name((row['R'],
                     row['G'], row['B']),colors), axis = 1) 
df_colors['color_name'] = df_colors.apply(lambda row : my_color_name((row['R'],
                     row['G'], row['B']),colors), axis = 1)
df_colors['color_desc'] = df_colors.apply(lambda row : GetColorDesc(row['hsv']), axis = 1) # color description: tone and temp


In [32]:
df_colors

,color,color_name,hex,R,G,B,hsv,tone,color_desc
0,Blue,Blue,#5d8aa8,93,138,168,"(204, 114, 204)",Air Force Blue (Raf),"(BRIGHT, COOL)"
1,Blue,Blue,#00308f,0,48,143,"(220, 255, 220)",Air Force Blue (Usaf),"(BRIGHT, COOL)"
2,Blue,Blue,#72a0c1,114,160,193,"(205, 104, 205)",Air Superiority Blue,"(BRIGHT, COOL)"
3,Red,Red,#a32638,163,38,56,"(351, 196, 351)",Alabama Crimson,"(BRIGHT, WARM)"
4,Blue,Blue,#f0f8ff,240,248,255,"(208, 15, 208)",Alice Blue,"(NEUTRAL, COOL)"
...,...,...,...,...,...,...,...,...,...
860,Orange,Orange,#ffae42,255,174,66,"(34, 189, 34)",Yellow Orange,"(BRIGHT, WARM)"
861,Yellow,Yellow,#ffef00,255,239,0,"(56, 255, 56)",Yellow (Process),"(BRIGHT, WARM)"
862,Yellow,Yellow,#fefe33,254,254,51,"(60, 204, 60)",Yellow (Ryb),"(BRIGHT, WARM)"
863,Blue,Blue,#0014a8,0,20,168,"(233, 255, 233)",Zaffre,"(BRIGHT, COOL)"


In [33]:
df_colors = df_colors.drop('color',axis=1)

In [34]:
df_colors.to_csv("new_colors.csv",index=False)